<a href="https://colab.research.google.com/github/K4cp3rski/ML_FUW/blob/master/Cybi%C5%84ski_02_zadanie_domowe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praca domowa I, zadanie II





## Treść

### Wstęp fabularny

Wyobraź sobie, że jesteś pracownikiem w firmie sprzedającej kompleksową usługę tworzenia wizerunków medialnych. Oddział, w którym pracujesz obsługuje ważnego klienta działającego w branży gier i usług cyfrowych.

Twoim zadaniem jest przygotować model uczenia maszynowego, który określać będzie nastawienie emocjonalne postów z Twittera. Zespół odpowiadający za zbieranie danych właśnie dostarczył zestaw danych dla Ciebie.

Do tej pory klasyfikowaniem nastrojów z twittów zajmował się zespół ekspertów. Rozwiązanie takie jest bardzo wolne i drogie, a dokładność ekspertów wynosi tylko 95%. Dlatego zarząd firmy zlecił wdrożenie modelu uczenia maszynowego.

Twój model stanowić będzie jedynie część większego produktu oferowanego przez Twoją firmę. Wyniki Twojego modelu będą bezpośrednio wykorzystywane przez następny zespół, którego zadaniem jest przygotować kolejny model uczenia maszynowego przewidujący reakcje opinii publicznej na posty klienta.

Prace zespołu, który korzystać będzie z Twojego modelu są już bardzo zaawansowane, dlatego nie może on pozwolić sobie na żadne dodatkowe zmiany w swoim projekcie. Absolutnie konieczne jest, aby Twój model przyporządkowywał posty do jednej z trzech klas 'Positive', 'Negative', 'Neutral' lub analogicznych. Posty nie na temat powinny być klasyfikowane jako 'Neutral'.

Notebook z Twoim projektem będzie oglądał Twój szef, więc koniecznie zadbaj, żeby znalazły się w nim najważniejsze przemyślenia, a rysunki były ładne.

Powodzenia 🦾

### Polecenia

1. Wstępna obróbka danych:

 - załaduj zbiór treningowy i testowy,
 - usuń wiersze o brakujących elementach,
 - w kolumnie `sentiment` zamień wartości `'Irrelevant'` na `'Neutral'`.

1. Wykonaj wizualizacje danych:

 - histogram tematów twittów (`entity`),
 - histogram nastawień (`sentiment`),
 - najczęściej padających słów w treści twittów (`content`).

1. Przygotuj dane:

 - przygotuj zbiór cech poprzez wektoryzacje kolumny `content`, 
 - przygotuj etykiety poprzez zakodowanie tekstowych wartości w kolumnie `sentiment` do postaci liczbowej.

  Następnie wytrenuj naiwny model bayesowski. Sprawdź działanie modelu na kilku własnoręcznie napisanych wiadomościach. 

1. Wytrenuj modele:
 - naiwny bayesowski,
 - liniowy SVM,
 - regresji logistycznej,
 - drzewo decyzyjne.

  Sprawdź model na danych treningowych (walidacja krzyżowa) i testowych, następnie wybierz najlepszy model. Uzasadnij swój wybór.
  
1. Zespół ekspertów ręcznie klasyfikuje dane z dokładnością 95%. Porównaj z nimi swój model i napisz jakie są przewagi Twojego modelu.

### Zbiór danych

Zbiór danych został przygotowany na podstawie zbioru [Twitter Sentiment Analysis](https://www.kaggle.com/jp797498e/twitter-entity-sentiment-analysis) i składa się z dwóch plików:
-  `twitter_training.csv` - zbiór treningowy,
- `twitter_validation.csv` - zbiór testowy.

Archiwum z plikami można pobrać z [dysku google](https://drive.google.com/file/d/1sw2vA87fmAI5V5Xl9k-PCSdN5XwydhOB/view?usp=sharing) lub odkomentowując poniższe linie:

In [334]:
# ! pip install gdown
# ! gdown https://drive.google.com/uc?id=1sw2vA87fmAI5V5Xl9k-PCSdN5XwydhOB
# ! unzip twitter.zip

In [335]:
# !pip install wordcloud

## Rozwiązanie

In [336]:
import pandas as pd
import numpy as np
import seaborn as sns

import sklearn
print('Zainstalowana wersja scikit-learn: {}.'.format(sklearn.__version__))

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10) # aby wykresy w Colabie były większe

import numpy as np
from scipy import diag, interp
from itertools import cycle

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import metrics

Zainstalowana wersja scikit-learn: 1.0.1.


In [337]:
train_data = pd.read_csv('twitter_training.csv')
test_data = pd.read_csv('twitter_validation.csv')

In [338]:
# # Wizualizacja danych

# f = sns.countplot(x='sentiment', data=train_data)
# h = sns.catplot(y='entity', hue='sentiment', data=train_data, kind='count', height=20, aspect=1)
# g = sns.catplot(x='sentiment', col='entity' , col_wrap=4, kind='count', data=train_data, height=4.5, aspect=1.2)

# (g.set_axis_labels("", "Tweet count")
# .set_xticklabels(["Positive", "Neutral", "Negative", 'Irrelevant'])
# .set_titles("{col_name}")
# .despine(left=True))  

Jak widzimy globalnie (Obrazek 1) klasy są w miare zbalansowane, liczebność nie różni się więcej jak 2-krotnie. W związku z tym wydaje się, że nie ma potrzeby sztucznego wyrównywania ich liczebności i można przejść do dalszego etapu preprocessingu danych

In [339]:
# Przydzielamy klasom integerowe labelki

# Neutral = Class 0
train_data.loc[train_data['sentiment'] == 'Positive', 'sentiment'] = 0
# Positive = Class 1
train_data.loc[train_data['sentiment'] == 'Neutral', 'sentiment'] = 1
# Negative = Class 2
train_data.loc[train_data['sentiment'] == 'Negative', 'sentiment'] = 2
# Irrelevant = Class 0
train_data.loc[train_data['sentiment'] == 'Irrelevant', 'sentiment'] = 0

# A teraz to samo dla zbiotu testowego

# Neutral = Class 0
test_data.loc[test_data['sentiment'] == 'Positive', 'sentiment'] = 0
# Positive = Class 1
test_data.loc[test_data['sentiment'] == 'Neutral', 'sentiment'] = 1
# Negative = Class 2
test_data.loc[test_data['sentiment'] == 'Negative', 'sentiment'] = 2
# Irrelevant = Class 0
test_data.loc[test_data['sentiment'] == 'Irrelevant', 'sentiment'] = 0

In [340]:
# print(train_data['sentiment'].unique())
# print(test_data['sentiment'].unique())

In [341]:
train_data = train_data.drop(columns='id')
test_data = test_data.drop(columns='id')

train_data = train_data.rename(columns={'sentiment':'Class'})
test_data = test_data.rename(columns={'sentiment':'Class'})

# Pozbywamy się wierszy z niepełnymi informacjani (NaN)
train_data = train_data.dropna(axis='rows', how='all', thresh=int(train_data.shape[1]))
test_data = test_data.dropna(axis='rows', how='all', thresh=int(test_data.shape[1]))

In [342]:
# # Zobaczmy sobie najczęściej występujące słowa w klasach słów pozytywnych, neutralnych, negatywnych i irrelevant

# import wordcloud
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# # Neutralne tweety
# neutral_list = list(map(str, train_data [train_data['Class'] == 0]['content']))

# neutral_words = " ".join(neutral_list)
# neutral_plot = WordCloud(width = 512, height = 512).generate(neutral_words)

# # Pozytywne tweety
# positive_list = list(map(str, train_data [train_data['Class'] == 1]['content']))

# positive_words = " ".join(positive_list)
# positive_plot = WordCloud(width = 512, height = 512).generate(positive_words)

# # Negatywne tweety
# negative_list = list(map(str, train_data [train_data['Class'] == 2]['content']))

# negative_words = " ".join(negative_list)
# negative_plot = WordCloud(width = 512, height = 512).generate(negative_words)

# plt.figure(figsize=(10,8))

In [343]:
# images = [neutral_plot, positive_plot, negative_plot]
# image_names = ['neutral', 'positive', 'negative']


# plt.figure(figsize=(20,5))
# plt.suptitle('Most frequent words in tweets class-wise',y = 0.95, x = 0.4, weight='heavy', size='xx-large')
# columns = 4
# for i, image in enumerate(images):
#     plt.subplot(len(images) / columns + 1, columns, i + 1)
#     plt.axis('off')
#     plt.title(image_names[i])
#     plt.imshow(image)

In [344]:
from sklearn.model_selection import train_test_split

X_train = train_data.drop(columns='Class')
y_train = np.asarray(train_data['Class']).reshape(-1, 1).ravel()
y_train = y_train.astype('int')

X_test = test_data.drop(columns='Class')
y_test = np.asarray(test_data['Class']).reshape(-1, 1).ravel()
y_test = y_test.astype('int')

Teraz jak już mamy zwizualizowane dane, to przekodujmy je na język zrozumiały przez maszynę, tj. wektory cech.

 W tym celu użyjemy funkcji *CountVectorizer*

In [345]:
entities_train = train_data['entity']
contents_train = train_data['content']

entities_test = test_data['entity']
contents_test = test_data['content']

In [346]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack

# Dopasowanie i wektoryzowanie dla danych treningowych

vectorizer_text = CountVectorizer() # stwórz instancje obiektu CountVectorizer dla kodowania tekstu tweetów
vectorizer_instances = CountVectorizer() # stwórz instancje obiektu CountVectorizer dla kodowania instancji
entities_train = vectorizer_instances.fit_transform(entities_train) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna instnacji)
contents_train = vectorizer_text.fit_transform(contents_train) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)

X_train = hstack((entities_train, contents_train))

# Wektoryzowanie i przetransformowanie danych testowych korzystając ze słownika stworzonego na bazie danych treningowych

entities_test = vectorizer_instances.transform(entities_test) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna instnacji)
contents_test = vectorizer_text.transform(contents_test) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)

X_test = hstack((entities_test, contents_test))


In [347]:
print("Dane treningowe: n_samples: %d, n_features: %d" % X_train.shape)

Dane treningowe: n_samples: 73996, n_features: 31100


Odwrotne mapowanie cech na słowa

In [348]:
# Dla kolumny z tekstem tweetów

feature_names_text = vectorizer_text.get_feature_names_out()
feature_names_text = np.asarray(feature_names_text)

# Dla kolumny z tematami

feature_names_instances = vectorizer_instances.get_feature_names_out()
feature_names_instances = np.asarray(feature_names_instances)

---
### Tworzymy instancję klasyfikatora MultinomialNB

In [349]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

---
### Sprawdźmy na zbiorze testowym

Robimy predykcję dla X_test

In [350]:
y_pred = clf.predict(X_test) # obliczamy predykcję dla tekstów ze zbioru testowego

In [351]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accur = accuracy_score(y_test, y_pred) # dokładność
print("Dokładność: %0.3f" % accur)
print("Classification report:") # wypisz raport klasyfikacji 

print(classification_report(y_test, y_pred))

print("Macierz błędów") # wypisz macierz (confusion matrix)
print(confusion_matrix(y_test, y_pred))

print("\nPrecyzja wyliczona built-in method = {}".format(clf.score(X_test, y_test)))

Dokładność: 0.831
Classification report:
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       449
           1       0.88      0.74      0.80       285
           2       0.79      0.82      0.81       266

    accuracy                           0.83      1000
   macro avg       0.83      0.82      0.82      1000
weighted avg       0.83      0.83      0.83      1000

Macierz błędów
[[402  17  30]
 [ 48 210  27]
 [ 34  13 219]]

Precyzja wyliczona built-in method = 0.831


### Możemy nasz klasyfikator ulepszyć, w tym celu trzeba poddać dane stemmingowi

Jednak jako, że jedna z naszych kolumn zawiera tylko nazwy gier/tematów o których jest pisane, to stemmingowi chcemy poddać jedynie kolumnę z treścią tweetów, bo to tam szukanie słów o wspólnych korzeniach znaczeniowych będzie mieć znaczenie

In [352]:
# Importy niezbędnych rzeczy

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [353]:
stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [356]:
entities_train = train_data['entity']
contents_train = train_data['content']

entities_test = test_data['entity']
contents_test = test_data['content']

# Dopasowanie i wektoryzowanie dla danych treningowych wraz ze stemmingiem

vectorizer_text = CountVectorizer(analyzer=stemmed_words) # stwórz instancje obiektu CountVectorizer dla kodowania tekstu tweetów
vectorizer_instances = CountVectorizer() # stwórz instancje obiektu CountVectorizer dla kodowania instancji
entities_train = vectorizer_instances.fit_transform(entities_train) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna instnacji)
contents_train = vectorizer_text.fit_transform(contents_train) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)

X_train = hstack((entities_train, contents_train))

# Wektoryzowanie i przetransformowanie danych testowych korzystając ze słownika stworzonego na bazie danych treningowych

entities_test = vectorizer_instances.transform(entities_test) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna instnacji)
contents_test = vectorizer_text.transform(contents_test) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)

X_test = hstack((entities_test, contents_test))



# <--- !! ---> #



# # Odwrotne mapowanie tweetów na słowa

# Dla kolumny z tekstem tweetów

feature_names_text = vectorizer_text.get_feature_names_out()
feature_names_text = np.asarray(feature_names_text)

# Dla kolumny z tematami

feature_names_instances = vectorizer_instances.get_feature_names_out()
feature_names_instances = np.asarray(feature_names_instances)



# <--- !! ---> #



# # # # Klasyfikator

# clf = MultinomialNB()
# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# accur = accuracy_score(y_test, y_pred) # dokładność
# print("Dokładność: %0.3f" % accur)
# print("Classification report:") # wypisz raport klasyfikacji 
# print(classification_report(y_test, y_pred))

# print("Macierz błędów") # wypisz macierz (confusion matrix)
# print(confusion_matrix(y_test, y_pred))

# print("Słowa, które z największą pewnością wskazują maszynie, że wiadomość jest pozytywna:")
# top10 = np.argsort(clf.coef_[0])[-10:]
# bottom10 = np.argsort(clf.coef_[0])[:10]
# print(feature_names_text[top10])

# print("Słowa najmniej istotne przy klasyfikacji:")
# print(feature_names_text[bottom10])

Dokładność: 0.797
Classification report:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       449
           1       0.85      0.69      0.76       285
           2       0.73      0.79      0.76       266

    accuracy                           0.80      1000
   macro avg       0.80      0.78      0.79      1000
weighted avg       0.80      0.80      0.80      1000

Macierz błędów
[[389  20  40]
 [ 51 197  37]
 [ 40  15 211]]


In [359]:
# # # Klasyfikator

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accur = accuracy_score(y_test, y_pred) # dokładność
print("Dokładność: %0.3f" % accur)
print("Classification report:") # wypisz raport klasyfikacji 
print(classification_report(y_test, y_pred))

print("Macierz błędów") # wypisz macierz (confusion matrix)
print(confusion_matrix(y_test, y_pred))

Dokładność: 0.797
Classification report:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       449
           1       0.85      0.69      0.76       285
           2       0.73      0.79      0.76       266

    accuracy                           0.80      1000
   macro avg       0.80      0.78      0.79      1000
weighted avg       0.80      0.80      0.80      1000

Macierz błędów
[[389  20  40]
 [ 51 197  37]
 [ 40  15 211]]
